In [ ]:
!pip install nibiru --quiet

# Introduction to py-sdk

The python sdk allows you to create queries and transaction to a Nibiru chain.

It allows to interact with all of the modules and can be leveraged to automate trading strategies or monitor them.

This notebook will guide you on opening and closing positions.

In [ ]:
import json
import requests
import time
import nibiru
import nibiru.msg

Let's set up the chain to which we will be connecting to. If you have a preferred endpoint you can update these values to connect to it. Make sure that the node have the port opens for 1317, 26657 and 9090.

In [ ]:
pair = "ubtc:unusd"

Next, let's make sure that the transaction we send are synchronous. This way if no error is raised when running the execute_msgs method, we are sure that the transaction was successful.

In [ ]:
network = nibiru.Network.testnet(2)

In [ ]:
tx_config = nibiru.TxConfig(tx_type=nibiru.TxType.BLOCK, gas_multiplier=3)
agent = (
    nibiru.Sdk.authorize().with_network(network).with_config(tx_config)
)


Our trader don't have any fund, so if we try to open a position it will fail saying that the address does not exist (never registered with the auth module of the chain).

Let's use the faucet to give some unusd and unibi to the trader

In [ ]:
response = requests.post(
    "https://faucet.testnet-2.nibiru.fi/",
    json={
        "address": agent.address,
        "coins": ["10000000unibi", "100000000000unusd"],
    },
)
print(response.content)

We can check the new balance of the trader:

In [ ]:
time.sleep(10)
agent.query.get_bank_balances(agent.address)

Now with those fresh token, we can open a long position

In [ ]:
_ = agent.tx.execute_msgs(
    nibiru.msg.perp.MsgOpenPosition(
        sender=agent.address,
        pair="ubtc:unusd",
        side=nibiru.Side.BUY,
        quote_asset_amount=10,
        leverage=10,
        base_asset_amount_limit=0,
    )
)


In [ ]:
def print_position():
    print(
        json.dumps(
            agent.query.perp.position(trader=agent.address, pair="ubtc:unusd"),
            indent=4,
        )
    )


print_position()

We can then add margin to our position

In [ ]:
tx_output = agent.tx.execute_msgs(
    nibiru.msg.perp.MsgAddMargin(
        sender=agent.address,
        pair=pair,
        margin=nibiru.Coin(10, "unusd"),
    )
)


In [ ]:
print_position()


We can then remove it

In [ ]:
tx_output = agent.tx.execute_msgs(
    nibiru.msg.perp.MsgRemoveMargin(
        sender=agent.address,
        pair=pair,
        margin=nibiru.Coin(5, "unusd"),
    )
)


In [ ]:
print_position()


We can then close totally the position

In [ ]:
tx_output = agent.tx.execute_msgs(
    nibiru.msg.perp.MsgClosePosition(sender=agent.address, pair=pair)
)

## Multi-transactions

We can build multiple messages into a single transactions to be sure that they are executed consecutively.

It can be useful for example to send tokens after removing margins from a position

Lets create a transction with:
- Agent open a position
- Agent remove some margin from the position
- Agent send this margin to a random address
- Agent close the position

In [ ]:
_ = agent.tx.execute_msgs(
    [
        nibiru.msg.perp.MsgOpenPosition(
            sender=agent.address,
            pair="ubtc:unusd",
            side=nibiru.Side.BUY,
            quote_asset_amount=100,
            base_asset_amount_limit=0,
            leverage=5,
        ),
        nibiru.msg.perp.MsgRemoveMargin(
            sender=agent.address,
            pair=pair,
            margin=nibiru.Coin(5, "unusd"),
        ),
        nibiru.msg.bank.MsgSend(
            from_address=agent.address,
            to_address="nibi1g39urhrvycnrc8y99xmqm7udr6fltjs3stnkxp", # Random address
            coins=[nibiru.Coin(5, "unusd")],
        ),
        nibiru.msg.perp.MsgClosePosition(sender=agent.address, pair=pair),
    ]
)